In [1]:
import pandas as pd
from collections import Counter
import numpy as np
import sys
import random
import functions
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from functions import *
from sklearn.model_selection import train_test_split
from pylab import rcParams
from sklearn.decomposition import PCA, FastICA
from sklearn.metrics import r2_score
import tensorflow as tf
import os
import datetime as dt
from tqdm import tnrange, tqdm_notebook, tqdm
from sklearn.metrics import r2_score


In [2]:
def InitSettings():
    param = dict()

    param['n_trees']= 500
    param['eta']= 0.005
    param['max_depth']= 4
    param['subsample']= 0.95
    param['objective']= 'reg:linear'
    param['eval_metric']= 'rmse'
    param['silent']= 1
    param['lambda']=1.0
    param ['alpha']=0.0 #[default=0]
    param['seed'] = 21

    others = dict()
    others['flagcat']=False
    others['flagcentering']=1
    others['cut']=0.075
    others['RemoveID']=False
    others['n_comp']=10
    others['scaleID']=0
    others['num_boost_rounds'] = 5500
    others['testfraction']=0.05
    others['removecatfromPCA']=False
    others['dumpresidual']=False
    others['removeoutliers']=False
    others['Ntry']=100
    others['randomsample']=True
    others['addlabel']=True
    allparams=dict()
    allparams['xgb']=param
    allparams['others']=others
    allparams['columns_for_remove']=[]
    return allparams

In [3]:
df_train,df_test,catcol=LoadandCleanData(True,1,0.0,2)
                                         

In [4]:
columnsPCA=list()
for i in df_test.columns:
    if i in catcol:
        continue
    columnsPCA.append(i)

In [5]:
df_train,df_test=AddLabel(df_train,df_test)

In [6]:
df_train,df_test=DoPCAICA(df_train,df_test,InitSettings(),columnsPCA)

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [7]:
touse=list()
for i in df_train.columns:
    if i.find("_")>-1:
        touse.append(i)

In [8]:
N=len(touse)

In [9]:
tf.reset_default_graph()
sess = tf.Session()

In [10]:


X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")
Xtest = tf.placeholder(tf.float32, name="Xtest")
Ytest = tf.placeholder(tf.float32, name="Ytest")

w1 = tf.Variable(tf.truncated_normal([504,30], stddev=0.00001), name="weights1")
b1=tf.Variable(tf.constant(0.0,shape=[30]), name="bias1")

w2 = tf.Variable(tf.truncated_normal([30,1], stddev=0.00001), name="weights2")
b2=tf.Variable(tf.constant(0.0,shape=[1]), name="bias2")

w3 = tf.Variable(tf.truncated_normal([30,1], stddev=0.00001), name="weights3")

#w4=tf.Variable(tf.truncated_normal([30,1], stddev=0.00001), name="weights4")
Y1= tf.nn.relu(tf.matmul(X,w1)+b1)
Y_predicted=tf.matmul(tf.square(Y1),w3)+tf.matmul(Y1,w2)+b2

step=tf.square(Y - Y_predicted,name="step")
chi2= tf.reduce_mean(step, name="loss")


global_step = tf.placeholder(tf.float32,name="Nstep")
starter_learning_rate = 0.05
learning_rate = starter_learning_rate*tf.exp(-1.0*global_step/500.0) 
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(chi2)


tf.summary.histogram("weights1", w1)
tf.summary.histogram("biases1", b1)
tf.summary.histogram("weights2", w2)
tf.summary.histogram("biases2", b2)
tf.summary.histogram("weights3", w3)
#tf.summary.histogram("weights4", w4)

tf.summary.histogram("Y", Y_predicted)
tf.summary.scalar("chi2", chi2)
tf.summary.scalar("rate", learning_rate)


Y1test= tf.nn.relu(tf.matmul(Xtest,w1)+b1)
Y_predictedtest=tf.matmul(tf.square(Y1test),w3)+tf.matmul(Y1test,w2)+b2


step=tf.square(Ytest - Y_predictedtest,name="steptest")
chi2test= tf.reduce_mean(step, name="losstest")
tf.summary.scalar("chi2logs", chi2test)

summ = tf.summary.merge_all()

In [11]:
inputdata=df_train.drop("y",axis=1).as_matrix()

In [12]:
type(inputdata),inputdata.shape

(numpy.ndarray, (4209, 504))

In [13]:
inputy=df_train['y'].as_matrix()

In [14]:
inputdata,testdata,inputy,testy=train_test_split(inputdata,inputy,test_size=0.1,random_state=21)

In [15]:
inputdata.shape,testdata.shape

((3788, 504), (421, 504))

In [ ]:
LOGDIR="./tf/{}/".format(dt.datetime.now().strftime('%Y%m%d_%H_%M_%S_'))
saver = tf.train.Saver()
#sess.run(tf.global_variables_initializer())
writer = tf.summary.FileWriter(LOGDIR)
writer.add_graph(sess.graph)

In [ ]:
Ntrain=len(inputdata)
Ntest=len(testdata)
#with tf.Session() as sess:
# Step 7: initialize the necessary variables, in this case, w and b
sess.run(tf.global_variables_initializer())
# Step 8: train the model
for j in tqdm(range(4000)): # run 100 epochs
    sess.run(optimizer,feed_dict={X:inputdata, Y:inputy.reshape(Ntrain,1),global_step:j}) 
    if j%5==0:
        s = sess.run(summ, feed_dict={X:inputdata, Y:inputy.reshape(Ntrain,1),Xtest:testdata, Ytest:testy.reshape(Ntest,1),global_step:j})
        writer.add_summary(s, j) 
        saver.save(sess, os.path.join(LOGDIR, "model.ckpt"), j)


 80%|████████  | 3210/4000 [02:16<00:30, 25.53it/s]

In [ ]:
Ytrain,Ytest=sess.run([Y_predicted,Y_predictedtest],feed_dict={X:inputdata, Y:inputy.reshape(Ntrain,1),Xtest:testdata, Ytest:testy.reshape(Ntest,1)})

In [ ]:
len(testdata)

In [ ]:
#[loss1,step1,b1]=sess.run([loss,step,b], feed_dict={X:inputdata, Y:inputy.reshape(N,1)})
sess.close()

In [ ]:
Ytrainv=[i[0] for i in Ytrain]
Ytestv=[i[0] for i in Ytest]

In [ ]:
len(testy)

In [ ]:
r2_score(Ytestv, testy)

In [ ]:
plt.hist(Ytrestv,bins=np.mgrid[-5:5:0.2],color="red",histtype="step",normed=True)
plt.hist(Ytrainv,bins=np.mgrid[-5:5:0.2],color='blue',histtype="step",normed=True)
plt.hist(df_train["y"].values,bins=np.mgrid[-5:5:0.2],color='green',histtype="step",normed=True)

In [ ]:
plt.plot(Ytrainv,df_train["y"].values,"go")

In [ ]:
timestamp=dt.datetime.now().strftime('%Y%m%d_%H_%M_%S')
output = pd.DataFrame({'id': df_test2['ID'].values, 'y': Ytrestv})
output.to_csv('TF{}.csv'.format(timestamp),index=False)

In [ ]:
df_test['ID'].values

In [ ]:
df_train2,df_test2=LoadandCleanData(True)

In [ ]:
df_test2

In [ ]:
type( writer)